In [9]:
from flask import Flask, render_template, request, jsonify
import numpy as np
import librosa
import os
from tensorflow.keras.models import load_model

app = Flask(__name__)

# Load the trained model
model = load_model("cnn_model.keras")

# Load label encoder
with open("label_encoder_classes.npy", "rb") as f:
    label_classes = np.load(f, allow_pickle=True)

# Function to extract MFCC features
def extract_mfcc(file_path):
    y, sr = librosa.load(file_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    mfcc = np.mean(mfcc, axis=1)
    return np.expand_dims(mfcc, axis=0)  # Add batch dimension

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file uploaded'}), 400

    file = request.files['file']
    
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    # Save the uploaded file temporarily
    file_path = os.path.join("temp.wav")
    file.save(file_path)

    # Extract MFCC features from the file
    mfcc_features = extract_mfcc(file_path)

    # Predict the speaker using the model
    prediction = model.predict(mfcc_features)
    predicted_label = np.argmax(prediction, axis=1)[0]

    # Convert predicted label to speaker name
    speaker_name = label_classes[predicted_label]

    # Return the result to the frontend
    return jsonify({'speaker': speaker_name})

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1